# 持久化选项
通过两种持久化方法：
* 快照
* 只追加文件 AOF append only file

## 快照持久化

### 创建快照的几种方法
* 客户端可以通过向redis发送BGSAVE命令来创建一个快照，此时会fork一个子进程来做存储的动作
* SAVE 不会用子进程，因此会使客户端不再相应命令，一般不常用，除非内存不够用
* 设置 save 配置选项: save 60 10000 指的是 当60秒内有10000次写入时，执行以下BGSAVE
* SHUTDOWN 会执行SAVE
* 当一个redis服务器连接另一个redis服务器，并向对方发送SYNC

### 几个适合快照的场景
* 个人开发
    * 例如设置 900 1 指的是900秒内如果有1条数据，则保存快照，如果没有，则在900秒后有一条数据进来就保存快照
* 对日志进行聚合计算
    * 首先确定日志丢失的可容忍程度，来决定存储快照的时间间隔
    * 需要恢复日志处理操作，知道存储快照时处理到了什么时候进度，此时需要知道被处理的文件和偏移量

In [1]:
import os
import redis
conn = redis.Redis()

In [3]:
def process_logs(conn, path, callback):
    # 获得当前的文档和offset
    current_file, offset = conn.mget('progress:file', 'progress:position')
    pipe = conn.pipeline()
    def update_progress():
        pipe.mset({
            'progress:file': fname,
            'progress:position': offset
        })
        pipe.execute()
    
    for fname in sorted(os.listdir(path)):
        if fname < current_file:
            continue
        with open(os.path.join(path, fname), 'rb') as inp:
            # 当是本文件，则从offset开始，否则从0开始
            if fname == current_file:
                inp.seek(int(offset, 10))
            else:
                offset = 0
            current_file = None
            for lno, line in enumerate(inp):
                callback(pipe, line)
                # offset指的是字符串的长度
                offset += int(offset) + len(line)
                if not (lno + 1) % 1000:
                    # 每1000行更新一下进度
                    update_grogress()
            update_progress()
            

### 大数据时
当数据量很大时，光创建子进程可能就要花去好多时间，如果内存不够用，可以选择关掉自动存快照，通过手动在不常用时间段 SAVE 的形式存储

## AOF持久化

* appendfsync: [always, everysec, no] 一般只推荐 everysec  
* AOF的问题就是，文件体积的大小，以及执行起来耗时间
    * 可以通过 BGREWRITEAOF 方法来去除冗余的命令
* 配置属性：auto-aof-rewrite-percentage / auto-aof-rewrite-min-size

# 复制

数据副本服务器处理客户端发送的读请求  
还需要检验硬盘写入

# 处理系统故障

* 验证快照和
* 主从切换/替换